<a href="https://colab.research.google.com/github/linhkid/gdg-codelab-25/blob/main/GDG_Gemma2.0_multiagent_funccall_vertex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>